In [1]:
import tensorflow as tf
import numpy as np
import random
from collections import deque
from sklearn.neighbors import NearestNeighbors
import os
import cv2
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image

# DQN hyperparameters
gamma = 0.95  # Discount factor for future rewards
epsilon = 1.0  # Exploration rate (starts with exploring)
epsilon_min = 0.01  # Minimum exploration rate
epsilon_decay = 0.995  # How quickly epsilon decreases
learning_rate = 0.001  # Learning rate for the neural network
batch_size = 64  # How many experiences to sample at a time

In [3]:

# Experience replay memory
memory = deque(maxlen=2000)

# Number of items to recommend at each step
n_actions = 5  # e.g., recommend 5 items

# Placeholder for item embeddings (content-based features)
item_embeddings = []

# Load ResNet50 for feature extraction
resnet_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet_model.trainable = False

# Function to extract image features (similar to content-based filtering)
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img).flatten()
    return features

In [9]:
# Define a simple DQN model
def create_dqn_model(input_shape, n_actions):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, input_shape=(input_shape,), activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(n_actions, activation='linear')  # Output Q-values for each action
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')
    return model


In [11]:
# Initialize DQN models (Q-network and target network)
input_shape = 2048  # Example shape after feature extraction 
model = create_dqn_model(input_shape, n_actions)
target_model = create_dqn_model(input_shape, n_actions)
target_model.set_weights(model.get_weights())  # Initially, both models are identical

In [13]:
import pickle

# Load precomputed embeddings from the pickle file
with open('embeddings.pkl', 'rb') as f:
    item_embeddings = pickle.load(f)

# Convert to numpy array if not already in that format
item_embeddings = np.array(item_embeddings)

# Ensure that you have the corresponding filenames loaded as well (for reference purposes)
filenames = []  # List to store image filenames for reference
for file in os.listdir('boys/Boys_resized'):  # Replace 'train_images' with the directory of your images
    filenames.append(os.path.join('boys/Boys_resized', file))

# Now you can use `item_embeddings` for your DQN training as usual


In [15]:
# Exploration vs exploitation: epsilon-greedy policy
def choose_action(state, epsilon):
    if np.random.rand() <= epsilon:
        return random.sample(range(n_actions), n_actions)  # Exploration: random recommendation
    q_values = model.predict(state)
    return np.argsort(q_values[0])[-n_actions:]  # Exploitation: recommend top-n items

In [19]:
# Replay memory for experience replay
def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

In [21]:
# Training DQN with experience replay
def replay():
    if len(memory) < batch_size:
        return
    minibatch = random.sample(memory, batch_size)
    for state, action, reward, next_state, done in minibatch:
        target = reward
        if not done:
            target += gamma * np.amax(target_model.predict(next_state)[0])
        target_f = model.predict(state)
        target_f[0][action] = target
        model.fit(state, target_f, epochs=1, verbose=0)
    global epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

In [23]:
# Function to update target network
def update_target_model():
    target_model.set_weights(model.get_weights())

In [25]:
# Simulate a right or left swipe
def simulate_swipe(action):
    # Assuming right swipe is +1 and left swipe is -1 reward
    return 1 if action == 'right' else -1

In [ ]:
from tqdm import tqdm
import time

# DQN Training with tqdm and early stopping
patience = 20  # Number of episodes with no improvement
best_reward = float('-inf')
no_improvement_count = 0

# Main loop (train over multiple episodes)
for episode in tqdm(range(15), desc="Episodes Progress", unit="episode"):
    start_time = time.time()  # Start timing the episode
    
    state = np.random.rand(1, input_shape)  # Initial user state (you can load the initial user preferences)
    done = False
    total_reward = 0
    
    for t in tqdm(range(20), desc=f"Episode {episode + 1} Steps", leave=False, unit="step"):
        action = choose_action(state, epsilon)
        
        # Simulate swipe (user feedback)
        reward = simulate_swipe('right' if np.random.rand() > 0.5 else 'left')
        next_state = np.random.rand(1, input_shape)  # Update the state with the next clothing recommendation
        
        remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward
        
        if done:
            break
        
        # Train the model
        replay()
        
        # Update target network every few episodes
        if episode % 10 == 0:
            update_target_model()
    
    # Measure time taken for the episode
    end_time = time.time()
    episode_time = end_time - start_time
    
    # Early stopping check
    if total_reward > best_reward:
        best_reward = total_reward
        no_improvement_count = 0
    else:
        no_improvement_count += 1
    
    # Print episode results
    print(f"Episode {episode + 1}, Total Reward: {total_reward}, Time Taken: {episode_time:.2f} seconds")
    
    # Early stopping condition
    if no_improvement_count >= patience:
        print(f"Early stopping at episode {episode + 1}, best reward: {best_reward}")
        break

# Save the model after training completes
model.save('dqn_fashion_recommender.h5')


Episode 1 Steps:   0%|                                 | 0/20 [00:00<?, ?step/s]
                                                                                

Episode 1, Total Reward: 0, Time Taken: 0.08 seconds



Episode 2 Steps:   0%|                                 | 0/20 [00:00<?, ?step/s]
                                                                                

Episode 2, Total Reward: 2, Time Taken: 0.01 seconds



Episode 3 Steps:   0%|                                 | 0/20 [00:00<?, ?step/s]
                                                                                

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━


Episode 13 Steps:  80%|██████████████████▍    | 16/20 [03:30<00:51, 12.80s/step]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━


Episode 13 Steps:  85%|███████████████████▌   | 17/20 [03:43<00:38, 12.71s/step]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━